# 3.1 Textlicher Teil Bebauungsplan – OCR+Embeddings (GPT-4o)

* Zero-Shot
* Chain-Of-Thought
* [LangChain Document Loader](https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/)

In [1]:
from utils.pdf2prompts import pdf2prompts
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

load_dotenv(find_dotenv())

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
    model_kwargs={"top_p": 0, "seed": 42},
    temperature=0,
)

In [77]:
# A) OCR via RapidOCR --> CNN/RNN != Transformer (LLM)
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
loader = PyMuPDFLoader(pdf_path, extract_images=True)
data = loader.load()
print(data[0])

Document(page_content='19.06.2000\nOviS\nG\n9833\nSCHRIFTLICHER TEIL (Teil B)\nBEBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"\nStadt LaIChInGen, GemarKUNG FeldSTeTfen, AlB-DoNaU-KreiS\nLageplan M 1: 500\nFur die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:\nBaugesetzbuch (BauGB)\nin der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).\nBaunutzungsverordnung (BauNvo)\n g   5        \nam 22.04.1993 (BGBl. 1. S. 466).\nPianzeichenverordnung 1990 (PlanzV 90)\nin der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 58).\nLandesbauordnung (LBO)\nin der Fassung der Bekanntmachung vom 08.08.1995 (GB1. S. 617).\nBisherige Festsetzungen:\nMit in Kraft treten dieses Bebauungsplanes treten im Geltungsbereich alle bisherigen\ngemeindlichen bauplanungsrechtlichen und bauordnungsrechtichen Festsetzungen auBer Kraft.', metadata={'source': '../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf', 'file_path': '

In [8]:
# B) OCR via GPT-4o
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"

async def page2text(page_prompt):
    msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Textinformationen aus dem schriftlichen Teil eines Bebauungsplans zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
            {
                "type": "text",
                "text": 'Extrahiere den kompletten Textinhalt. Ausgabe im XML-Format'
            },
            page_prompt
            ]
        ),
    ])
    return msg.content

prompts = pdf2prompts(pdf_path)
data_loaders = list(map(lambda prompt: page2text(prompt), prompts))
data = await asyncio.gather(*data_loaders)
print(data[0])